In [4]:
import numpy as np
import skimage.measure
# import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from keras import backend as K
import keras
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Input
import tensorflow as tf

In [7]:
plt.gray()
os.chdir('C:/Users/span/Documents/CNN/')

<Figure size 432x288 with 0 Axes>

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
)
        
val_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
# only 0 and 255 in seg
subfolder1 = "_0255/"
# subfolder1 = "/"
subfolder2 = "_0255/"

path1 = "02_data/train_image"+subfolder1
path2 = "02_data/train_mask"+subfolder2
path3 = "02_data/val_image"+subfolder1
path4 = "02_data/val_mask"+subfolder2

# # scale to 255 in val
# path1 = "02_data/train_image_re255/"
# path2 = "02_data/train_mask_re255/"
# path3 = "02_data/val_image_re255/"
# path4 = "02_data/val_mask_re255/"

batch_size1 = 16

train_image_generator = train_datagen.flow_from_directory(
    path1,
    color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

train_mask_generator = train_datagen.flow_from_directory(
    path2,
    color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

val_image_generator = val_datagen.flow_from_directory(
    path3,
    color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)


val_mask_generator = val_datagen.flow_from_directory(
    path4,
    color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

Found 472 images belonging to 1 classes.
Found 472 images belonging to 1 classes.
Found 203 images belonging to 1 classes.
Found 203 images belonging to 1 classes.


In [ ]:
def my_image_mask_generator(image_data_generator, mask_data_generator):
    train_generator = zip(image_data_generator, mask_data_generator)
    for (img, mask) in train_generator:
        yield (img, mask)
        
train_generator = my_image_mask_generator(train_image_generator, train_mask_generator)
val_generator = my_image_mask_generator(val_image_generator, val_mask_generator)

In [1]:
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, concatenate, UpSampling2D
from keras import backend as K
import keras
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Input
import tensorflow as tf

def res_identity(x, filters): 
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # third block activation used after adding the input
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  # x = Activation(activations.relu)(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

In [2]:
def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

In [3]:
def resnet50():

  input_im = Input(shape=(train_im.shape[1], train_im.shape[2], train_im.shape[3])) # cifar 10 images size
  x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 3rd stage

  x = res_conv(x, s=2, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))

  # 4th stage

  x = res_conv(x, s=2, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))

  # 5th stage

  x = res_conv(x, s=2, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))

  # ends with average pooling and dense connection

  x = AveragePooling2D((2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(len(class_types), activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model

In [4]:
# train model
model = resnet50()

NameError: name 'train_im' is not defined

In [23]:
inputs = keras.Input((sample_width, sample_height, 1))
conv1 = keras.layers.Conv2D(n_filters*2, (3, 3), activation='relu', padding='SAME')(inputs)
conv1 = keras.layers.Conv2D(n_filters*2, (3, 3), activation='relu', padding='SAME')(conv1)
conv1 = tf.reshape(conv1, tf.shape(conv1))
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
drop1 = keras.layers.Dropout(0.2)(pool1)

print(drop1.shape)

(None, 256, 256, 16)


In [24]:
conv2 = keras.layers.Conv2D(n_filters*4, (3, 3), activation='relu', padding='SAME')(drop1)
conv2 = keras.layers.Conv2D(n_filters*4, (3, 3), activation='relu', padding='SAME')(conv2)
conv2 = tf.reshape(conv2, tf.shape(conv2))
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
drop2 = keras.layers.Dropout(0.2)(pool2)

print(drop2.shape)

(None, 128, 128, 32)


In [25]:
conv3 = keras.layers.Conv2D(n_filters*8, (3, 3), activation='relu', padding='SAME')(drop2)
conv3 = keras.layers.Conv2D(n_filters*8, (3, 3), activation='relu', padding='SAME')(conv3)
conv3 = tf.reshape(conv3, tf.shape(conv3))
pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
drop3 = keras.layers.Dropout(0.3)(pool3)

print(drop3.shape)

(None, 64, 64, 64)


In [26]:
conv4 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(drop3)
conv4 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(conv4)
conv4 = tf.reshape(conv4, tf.shape(conv4))
pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
drop4 = keras.layers.Dropout(0.3)(pool4)

print(drop4.shape)

(None, 32, 32, 128)


In [27]:
conv5 = keras.layers.Conv2D(n_filters*32, (3, 3), activation='relu', padding='SAME')(drop4)
conv5 = keras.layers.Conv2D(n_filters*32, (3, 3), activation='relu', padding='SAME')(conv5)
conv5 = tf.reshape(conv5, tf.shape(conv5))
    
print(conv5.shape)

(None, 32, 32, 256)


In [28]:
conv5u = tf.keras.layers.Conv2DTranspose(filters=n_filters*16, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv5)
conv5u = tf.reshape(conv5u, tf.shape(conv5u))
up6 = keras.layers.concatenate([conv5u, conv4], axis=3)
up6 = keras.layers.Dropout(0.2)(up6)
conv6 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(up6)
conv6 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(conv6)
conv6 = tf.reshape(conv6, tf.shape(conv6))
    
print(conv5u.shape, conv4.shape, "\n", conv6.shape)

(None, 64, 64, 128) (None, 64, 64, 128) 
 (None, 64, 64, 128)


In [29]:
conv5u = keras.layers.Conv2D(n_filters*16, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(conv5))
conv5u = tf.reshape(conv5u, tf.shape(conv5u))
up6 = keras.layers.concatenate([conv5u, conv4], axis=3)
up6 = keras.layers.Dropout(0.2)(up6)
conv6 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(up6)
conv6 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(conv6)
conv6 = tf.reshape(conv6, tf.shape(conv6))
print(conv5u.shape, conv4.shape, "\n", conv6.shape)

(None, 64, 64, 128) (None, 64, 64, 128) 
 (None, 64, 64, 128)


In [30]:
conv6u = tf.keras.layers.Conv2DTranspose(filters=n_filters*8, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv6)
conv6u = tf.reshape(conv6u, tf.shape(conv6u))
up7 = keras.layers.concatenate([conv6u, conv3], axis=3)
up7 = keras.layers.Dropout(0.2)(up7)
conv7 = keras.layers.Conv2D(n_filters*8, (3, 3), activation='relu', padding='SAME')(up7)
conv7 = keras.layers.Conv2D(n_filters*8, (3, 3), activation='relu', padding='SAME')(conv7)
conv7 = tf.reshape(conv7, tf.shape(conv7))
    
print(conv6u.shape, conv3.shape, "\n", conv7.shape)

(None, 128, 128, 64) (None, 128, 128, 64) 
 (None, 128, 128, 64)


In [31]:
conv7u = tf.keras.layers.Conv2DTranspose(filters=n_filters*4, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv7)
conv7u = tf.reshape(conv7u, tf.shape(conv7u))
up8 = keras.layers.concatenate([conv7u, conv2], axis=3)
up8 = keras.layers.Dropout(0.2)(up8)
conv8 = keras.layers.Conv2D(n_filters*4, (3, 3), activation='relu', padding='SAME')(up8)
conv8 = keras.layers.Conv2D(n_filters*4, (3, 3), activation='relu', padding='SAME')(conv8)
conv8 = tf.reshape(conv8, tf.shape(conv8))
    
print(conv7u.shape, conv2.shape, "\n", conv8.shape)

(None, 256, 256, 32) (None, 256, 256, 32) 
 (None, 256, 256, 32)


In [10]:
conv8u = tf.keras.layers.Conv2DTranspose(filters=n_filters*2, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv8)
conv8u = tf.reshape(conv8u, tf.shape(conv8u))
up9 = keras.layers.concatenate([conv8u, conv1], axis=3)
up9 = keras.layers.Dropout(0.2)(up9)
conv9 = keras.layers.Conv2D(n_filters*2, (3, 3), activation='relu', padding='SAME')(up9)
conv9 = keras.layers.Conv2D(n_filters*2, (3, 3), activation='relu', padding='SAME')(conv9)
conv9 = tf.reshape(conv9, tf.shape(conv9))

print(conv8u.shape, conv1.shape, "\n", conv9.shape)

(None, 256, 256, 32) (None, 256, 256, 32) 
 (None, 256, 256, 32)


In [11]:
conv10 = keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)
conv10 = tf.reshape(conv10, tf.shape(conv10))
print(conv10.shape)

(None, 256, 256, 1)
